<a href="https://colab.research.google.com/github/yusufsahin/python-training-maro/blob/main/ders10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip -q install fastapi "uvicorn[standard]" sqlalchemy pydantic "pydantic[email]"

In [25]:
import sys
import os

# Explicitly create directories for the 'app' package structure
os.makedirs('app/core', exist_ok=True)
os.makedirs('app/db', exist_ok=True)
os.makedirs('app/users', exist_ok=True)

# Create empty __init__.py files to ensure Python recognizes these as packages
open('app/__init__.py', 'a').close()
open('app/core/__init__.py', 'a').close()
open('app/db/__init__.py', 'a').close()
open('app/users/__init__.py', 'a').close()

# Add the current directory to sys.path to make the 'app' module discoverable
sys.path.append(os.getcwd())

In [26]:
from fastapi.testclient import TestClient
from app.main import app

client = TestClient(app)

r = client.post("/users", json={"email": "a@b.com", "full_name": "Ada Lovelace"})
print(r.status_code, r.json())

r = client.get("/users/1")
print(r.status_code, r.json())

r = client.get("/users")
print(r.status_code, r.json())

ModuleNotFoundError: No module named 'app.main'

In [22]:
#%%writefile app/main.py
from fastapi import FastAPI
from app.core.config import settings
from app.db.init_db import init_db
from app.users.router import router as users_router

def create_app() -> FastAPI:
    app = FastAPI(title=settings.app_name)
    app.include_router(users_router)

    @app.on_event("startup")
    def _startup():
        init_db()

    return app

app = create_app()

ModuleNotFoundError: No module named 'app.core.config'

In [21]:
#%%writefile app/users/router.py
from fastapi import APIRouter, Depends
from sqlalchemy.orm import Session

from app.db.session import get_db
from app.users.schemas import UserCreate, UserOut
from app.users.service import UserService

router = APIRouter(prefix="/users", tags=["users"])
service = UserService()

@router.post("", response_model=UserOut, status_code=201)
def create_user(payload: UserCreate, db: Session = Depends(get_db)):
    return service.register_user(db, payload)

@router.get("/{user_id}", response_model=UserOut)
def get_user(user_id: int, db: Session = Depends(get_db)):
    return service.get_user(db, user_id)

@router.get("", response_model=list[UserOut])
def list_users(limit: int = 50, offset: int = 0, db: Session = Depends(get_db)):
    # list için service yazmak opsiyonel; kural yoksa repo da kullanılabilir
    from app.users.repository import UserRepository
    return UserRepository().list(db, limit=limit, offset=offset)

ModuleNotFoundError: No module named 'app.db.session'

In [27]:
#%%writefile app/users/service.py
from sqlalchemy.orm import Session
from fastapi import HTTPException, status
from app.users.repository import UserRepository
from app.users.schemas import UserCreate
from app.db.init_db import User

class UserService:
    def __init__(self, repo: UserRepository | None = None):
        self.repo = repo or UserRepository()

    def register_user(self, db: Session, data: UserCreate) -> User:
        existing = self.repo.get_by_email(db, str(data.email))
        if existing:
            raise HTTPException(
                status_code=status.HTTP_409_CONFLICT,
                detail="Email already exists",
            )
        return self.repo.create(db, data)

    def get_user(self, db: Session, user_id: int) -> User:
        user = self.repo.get_by_id(db, user_id)
        if not user:
            raise HTTPException(status_code=404, detail="User not found")
        return user

ModuleNotFoundError: No module named 'app.users.repository'

In [19]:
#%%writefile app/users/repository.py
from sqlalchemy.orm import Session
from sqlalchemy import select
from app.db.init_db import User
from app.users.schemas import UserCreate

class UserRepository:
    def get_by_email(self, db: Session, email: str) -> User | None:
        stmt = select(User).where(User.email == email)
        return db.execute(stmt).scalar_one_or_none()

    def get_by_id(self, db: Session, user_id: int) -> User | None:
        return db.get(User, user_id)

    def list(self, db: Session, limit: int = 50, offset: int = 0) -> list[User]:
        stmt = select(User).offset(offset).limit(limit)
        return list(db.execute(stmt).scalars().all())

    def create(self, db: Session, data: UserCreate) -> User:
        user = User(email=str(data.email), full_name=data.full_name)
        db.add(user)
        db.commit()
        db.refresh(user)
        return user

ModuleNotFoundError: No module named 'app.db.init_db'

In [18]:
#%%writefile app/users/schemas.py

from pydantic import BaseModel, ConfigDict, EmailStr

class UserCreate(BaseModel):
    email: EmailStr
    full_name: str

class UserOut(BaseModel):
    model_config = ConfigDict(from_attributes=True)  # ORM obj -> schema
    id: int
    email: EmailStr
    full_name: str

In [15]:
#%%writefile app/db/init_db.py

from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy import String

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "users"

    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    email: Mapped[str] = mapped_column(String(255), unique=True, index=True)
    full_name: Mapped[str] = mapped_column(String(255))

def init_db() -> None:
    from app.db.session import engine # Import engine here to avoid circular dependency
    Base.metadata.create_all(bind=engine)

In [14]:
#%%writefile app/db/session.py

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session
from app.core.config import settings

# SQLite + FastAPI'da çoklu thread senaryoları için sık kullanılan ayar
# (özellikle TestClient / threadpool ile).
engine = create_engine(
    settings.sqlite_url,
    connect_args={"check_same_thread": False},
    future=True,
)

SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False, future=True)

def get_db():
    db: Session = SessionLocal()
    try:
        yield db
    finally:
        db.close()

ModuleNotFoundError: No module named 'app.core.config'

In [12]:
from fastapi.testclient import TestClient
from app.main import app

client = TestClient(app)

r = client.post("/users", json={"email": "a@b.com", "full_name": "Ada Lovelace"})
print(r.status_code, r.json())

r = client.get("/users/1")
print(r.status_code, r.json())

r = client.get("/users")
print(r.status_code, r.json())

ModuleNotFoundError: No module named 'app.main'

In [30]:
#%%writefile app/core/config.py

from pydantic import BaseModel, ConfigDict

class Settings(BaseModel):
    model_config = ConfigDict(extra="ignore")
    app_name: str = "Demo API"
    sqlite_url: str = "sqlite:///./demo.db"

settings = Settings()